# Estimate Metalinks

In [1]:
import numpy as np
import pandas as pd
import scanpy as sc
import muon as mu
import liana as li

/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from liana.funcomics import neg_to_zero

In [3]:
import decoupler as dc

Test object

In [4]:
adata = sc.datasets.pbmc3k_processed()
adata = adata.raw.to_adata()

## Metabolite Enrichment

In [5]:
resource = pd.read_csv("liana/resource/metalinks_resource.csv")

In [6]:
resource = resource[['HMDB', 'Symbol']].rename(columns={'HMDB': 'ligand', 'Symbol': 'receptor'})

In [7]:
resource

,ligand,receptor
0,HMDB0000112,GABRB2
1,HMDB0001043,CLCN2
2,HMDB0015180,CLCN2
3,HMDB0000216,ADRA2B
4,HMDB0000068,ADRA2B
...,...,...
10160,HMDB0041130,MUSK
10161,HMDB0002496,MUSK
10162,HMDB0015397,MUSK
10163,HMDB0006284,MUSK


Process net

In [8]:
# replace direction with 1 or -1
met_net = pd.read_csv("liana/resource/metabolite_sets.csv")
met_net['direction'].replace({'degrading': -1, 'producing': 1}, inplace=True)

In [9]:
# NOTE: there are duplicates, Elias should append Sources
met_net = met_net.drop_duplicates(['HMDB', 'GENE', 'direction'])

In [10]:
# NOTE: there are sign-duplicates...
met_net = met_net.groupby(['HMDB', 'GENE']).aggregate({'direction': 'mean'}).reset_index()
# Remove any edge whose direction is not 1 or -1
met_net = met_net[met_net['direction'].isin([-1, 1])]

Metabolite Estimate

In [11]:
# dc.run_ulm(adata, net=met_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=2)
dc.run_wmean(adata, net=met_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=2, times=0)

In [12]:
met_est = li.fun.obsm_to_adata(adata, 'wmean_estimate')
met_est

AnnData object with n_obs × n_vars = 2638 × 714
    obs: 'n_genes', 'percent_mito', 'n_counts', 'louvain'
    uns: 'draw_graph', 'louvain', 'louvain_colors', 'neighbors', 'pca', 'rank_genes_groups'
    obsm: 'X_pca', 'X_tsne', 'X_umap', 'X_draw_graph_fr', 'wmean_estimate'
    obsp: 'distances', 'connectivities'

To MuData

In [13]:
mdata = mu.MuData({'metabolite':met_est, 'rna':adata})

Run LIANA

In [16]:
li.mt.cellphonedb(mdata,
                  groupby='louvain',
                  multi_kwargs=dict(x_mod='metabolite', y_mod='rna',
                                    x_transform = neg_to_zero,
                                    y_transform = False),
                  n_perms=None, use_raw=False, verbose=True,
                  resource=resource
                  )

Using `.X`!
Transforming metabolite using neg_to_zero
Using `.X`!
Using `.X`!
110 features of mat are empty, they will be removed.
Using provided `resource`.


/home/dbdimitrov/anaconda3/envs/spiana/lib/python3.10/site-packages/pandas/core/indexing.py:1728: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.


0.74 of entities in the resource are missing from the data.
Generating ligand-receptor stats for 2638 samples and 14318 features


In [19]:
mdata.uns['liana_res']

,ligand,ligand_complex,ligand_means,ligand_props,receptor,receptor_complex,receptor_means,receptor_props,source,target,lr_means
3419,HMDB0000169,HMDB0000169,0.011945,0.266667,FCER1A,FCER1A,1.583187,0.837838,Megakaryocytes,Dendritic cells,0.797566
3182,HMDB0000169,HMDB0000169,0.007202,0.453333,FCER1A,FCER1A,1.583187,0.837838,FCGR3A+ Monocytes,Dendritic cells,0.795194
2745,HMDB0000169,HMDB0000169,0.007171,0.335417,FCER1A,FCER1A,1.583187,0.837838,CD14+ Monocytes,Dendritic cells,0.795179
3302,HMDB0000169,HMDB0000169,0.005620,0.378378,FCER1A,FCER1A,1.583187,0.837838,Dendritic cells,Dendritic cells,0.794403
3070,HMDB0000169,HMDB0000169,0.003151,0.162338,FCER1A,FCER1A,1.583187,0.837838,NK cells,Dendritic cells,0.793169
...,...,...,...,...,...,...,...,...,...,...,...
3561,HMDB0001381,HMDB0001381,0.000000,0.502083,TBXA2R,TBXA2R,0.138629,0.133333,CD14+ Monocytes,Megakaryocytes,0.000000
2318,HMDB0000285,HMDB0000285,0.000000,0.952532,P2RY13,P2RY13,0.095123,0.133333,CD8 T cells,FCGR3A+ Monocytes,0.000000
2321,HMDB0000285,HMDB0000285,0.000000,0.952532,P2RX1,P2RX1,0.144821,0.193333,CD8 T cells,FCGR3A+ Monocytes,0.000000
2354,HMDB0000131,HMDB0000131,0.000000,0.136364,TGFBR2,TGFBR2,0.113607,0.160000,NK cells,FCGR3A+ Monocytes,0.000000


## Check Transporters

Transporter estimate

(Note worth it for this part)

In [ ]:
t_net = pd.read_csv("liana/resource/transporter_sets.csv")

In [ ]:
# when revirsible == True, else False
t_net['reversibility'] = t_net['reversibility']=='reversible'

In [ ]:
in_net = t_net[(t_net['transport_direction'] == 'in') | (t_net['reversibility'])]
in_net

In [ ]:

out_net = t_net[(t_net['transport_direction'] == 'out') | (t_net['reversibility'])]
out_net

In [ ]:
dc.run_ulm(adata, net=out_net, source='HMDB', target='GENE', weight='direction', use_raw=False, min_n=3)